#### 📌 **HİPERPARAMETRELERİ BURADAN AYARLA**

In [5]:
import os
import re
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from tensorflow.keras.models import load_model

In [6]:
def mammo_gorunumlerini_listele(kok_dizin):
    class_dirs = {
        'normal': ['Birad1'],
        'benign': ['Birad3'],
        'cancer': ['Birad4', 'Birad5'],
    }
    paths, labels = [], []
    for cls, subdirs in class_dirs.items():
        for sd in subdirs:
            dir_path = os.path.join(kok_dizin, sd)
            if not os.path.isdir(dir_path):
                continue
            for fname in sorted(os.listdir(dir_path)):
                if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                    paths.append(os.path.join(dir_path, fname))
                    labels.append(cls)
    return paths, labels

In [7]:
#chatgpt ile ayırdım
def read_grayscale_image(path, target_size):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    return img.astype(np.float32)

def normalize_image(img, method='minmax'):
    if method == 'minmax':
        return img / 255.0
    else:
        return (img - img.mean()) / (img.std() + 1e-8)

def load_and_preprocess(paths, labels, target_size, method='minmax'):
    X, y = [], []
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    for p, lbl in zip(paths, labels):
        img = read_grayscale_image(p, target_size)
        img = normalize_image(img, method)
        X.append(img[..., np.newaxis])
        y.append(label_map[lbl])
    return np.array(X), np.array(y)

 Metirkler**

In [8]:
if __name__ == '__main__':
    kok_dizin = os.path.join(os.getcwd(), "Project2")
    yollar, etiketler = mammo_gorunumlerini_listele(kok_dizin)
    X_test, y_test = load_and_preprocess(yollar, etiketler,target_size=(224, 224))
    model=load_model("model2_2.h5")
    T_m = 0.40
    T_b = 0.40
    probs = model.predict(X_test)
    classes = ['normal', 'benign', 'cancer']
    label_map = {'normal': 0, 'benign': 1, 'cancer': 2}
    preds = [2 if p[2] >= T_m else 1 if p[1] >= T_b else 0 for p in probs]
    print(classification_report(y_test, preds, target_names=classes))
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("Accuracy:", accuracy_score(y_test, preds))
    print("Precision:", precision_score(y_test, preds, average='macro'))
    print("Recall:", recall_score(y_test, preds, average='macro'))
    print("📌 F1 Score:", f1_score(y_test, preds, average='macro'))
    try:
        print("AUC:", roc_auc_score(pd.get_dummies(y_test), probs,
                                 multi_class='ovr', average='macro'))
    except:
        print("AUC hesaplanamadı.")

              precision    recall  f1-score   support

      normal       0.83      1.00      0.91        20
      benign       0.83      1.00      0.91        20
      cancer       1.00      0.60      0.75        20

    accuracy                           0.87        60
   macro avg       0.89      0.87      0.86        60
weighted avg       0.89      0.87      0.86        60

Confusion Matrix:
 [[20  0  0]
 [ 0 20  0]
 [ 4  4 12]]
Accuracy: 0.8666666666666667
Precision: 0.888888888888889
Recall: 0.8666666666666667
📌 F1 Score: 0.856060606060606
AUC: 0.97375
